In [1]:
import pandas as pd
import numpy as np
import statistics
pd.options.mode.chained_assignment = None

### Скачиваем данные

In [2]:
data = pd.read_csv("new_partners_cnt.csv")
data.drop(columns=['Unnamed: 0'], inplace=True)
data['date'] = pd.to_datetime(data['date'])
data

,delivery_area_id,date,partners_cnt,delay_rate
0,0,2021-04-01 10:00:00,4.0,0.111111
1,0,2021-04-01 11:00:00,4.0,0.000000
2,0,2021-04-01 13:00:00,1.0,0.000000
3,0,2021-04-01 14:00:00,1.0,0.000000
4,0,2021-04-01 15:00:00,1.0,0.000000
...,...,...,...,...
918392,592,2021-12-07 17:00:00,1.0,0.000000
918393,592,2021-12-07 18:00:00,1.0,0.000000
918394,592,2021-12-07 19:00:00,1.0,0.000000
918395,592,2021-12-07 20:00:00,1.0,0.000000


### Выбираем данные для новых смен

In [3]:
new_dates = data.loc[data['date'] > '2021-12-01']
new_dates_list = new_dates['date'].dt.date.unique()
for shift_time in range(4, 9):
    new_dates['shift_time-%d' % shift_time] = 0
new_dates

,delivery_area_id,date,partners_cnt,delay_rate,shift_time-4,shift_time-5,shift_time-6,shift_time-7,shift_time-8
2833,0,2021-12-01 10:00:00,5.0,0.0,0,0,0,0,0
2834,0,2021-12-01 11:00:00,6.0,0.0,0,0,0,0,0
2835,0,2021-12-01 12:00:00,5.0,0.0,0,0,0,0,0
2836,0,2021-12-01 13:00:00,3.0,0.0,0,0,0,0,0
2837,0,2021-12-01 14:00:00,3.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
918392,592,2021-12-07 17:00:00,1.0,0.0,0,0,0,0,0
918393,592,2021-12-07 18:00:00,1.0,0.0,0,0,0,0,0
918394,592,2021-12-07 19:00:00,1.0,0.0,0,0,0,0,0
918395,592,2021-12-07 20:00:00,1.0,0.0,0,0,0,0,0


### Рассчитываем смены исходя из числа нужных партнеров

In [5]:
areas = new_dates['delivery_area_id'].unique()
for area_id in [0]:
    for date in new_dates_list:
        cur_day = new_dates[(new_dates['delivery_area_id'] == area_id) & (new_dates['date'].dt.date == date)].copy()
        cur_day['partners_left'] = cur_day['partners_cnt']
        for hour in range(10, 22):
            future_partners = cur_day[cur_day['date'].dt.hour >= hour]['partners_left'].tolist()
            if not future_partners:
                continue
            required_partners = max(future_partners[0], 0)
            if not required_partners:
                continue
            try:
                shift_length = max(future_partners.index(0), 4)
            except:
                shift_length = min(max(len(future_partners), 4), 8)
            
            cur_day.loc[cur_day['date'].dt.hour == hour, 'shift_time-%d' % shift_length] = required_partners
            cur_day.loc[(cur_day['date'].dt.hour >= hour) & (cur_day['date'].dt.hour <= hour + shift_length), 'partners_left'] -= required_partners
        for shift_length in range(4, 9):
            new_dates.loc[(new_dates['delivery_area_id'] == area_id) & (new_dates['date'].dt.date == date), 'shift_time-%d' % shift_length] = cur_day['shift_time-%d' % shift_length]
new_dates.head(10)

,delivery_area_id,date,partners_cnt,delay_rate,shift_time-4,shift_time-5,shift_time-6,shift_time-7,shift_time-8
2833,0,2021-12-01 10:00:00,5.0,0.0,0,0,0,0,5
2834,0,2021-12-01 11:00:00,6.0,0.0,1,0,0,0,0
2835,0,2021-12-01 12:00:00,5.0,0.0,0,0,0,0,0
2836,0,2021-12-01 13:00:00,3.0,0.0,0,0,0,0,0
2837,0,2021-12-01 14:00:00,3.0,0.0,0,0,0,0,0
2838,0,2021-12-01 15:00:00,4.0,0.0,0,0,0,0,0
2839,0,2021-12-01 16:00:00,4.0,0.0,0,0,0,0,0
2840,0,2021-12-01 17:00:00,3.0,0.0,0,0,0,0,0
2841,0,2021-12-01 18:00:00,5.0,0.0,0,0,0,0,0
2842,0,2021-12-01 19:00:00,3.0,0.0,3,0,0,0,0


### Для каждой зоны доставки и каждого времени колонка shift_time-i для i от 4 до 8 означает число человек, которые должны выйти в это время на смену длиной i часов

In [6]:
new_dates.to_csv('final_shifts.csv')